In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict
from copy import deepcopy
import json
import os
import warnings

import k3d
import yaml
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
from joblib import Parallel, delayed

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-f6pu27hs because the default path (/home/user/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [3]:
from sharpf.utils.abc_utils.hdf5.dataset import Hdf5File, PreloadTypes
import sharpf.data.datasets.sharpf_io as io 

In [4]:
def display_sharpness(mesh=None, plot_meshvert=True,
                      samples=None, samples_distances=None,
                      sharp_vert=None, sharp_curves=None,
                      directions=None, directions_width=0.0025,
                      samples_color=0x0000ff, samples_psize=0.002, 
                      mesh_color=0xbbbbbb, meshvert_color=0x666666, meshvert_psize=0.0025,
                      sharpvert_color=0xff0000, sharpvert_psize=0.0025,
                      sharpcurve_color=None, sharpcurve_width=0.0025,
                      as_image=False, plot_height=768,
                      cmap=k3d.colormaps.matplotlib_color_maps.coolwarm_r):
    
    plot = k3d.plot(height=plot_height)
    
    if None is not mesh:
        k3d_mesh = k3d.mesh(mesh.vertices, mesh.faces, color=mesh_color)
        plot += k3d_mesh

        if plot_meshvert:
            k3d_points = k3d.points(mesh.vertices, 
                                    point_size=meshvert_psize, color=meshvert_color)
            plot += k3d_points
            k3d_points.shader='flat'

    if None is not samples:
        colors = None
        if None is not samples_distances:
            max_dist = 1.0

            colors = k3d.helpers.map_colors(
                samples_distances, cmap, [0, max_dist]
            ).astype(np.uint32)
            k3d_points = k3d.points(samples, point_size=samples_psize, colors=colors)
        else:
            k3d_points = k3d.points(samples, point_size=samples_psize, color=samples_color)
        plot += k3d_points
        k3d_points.shader='flat'
        
        if None is not directions:
            vectors = k3d.vectors(
                samples,
                directions * samples_distances[..., np.newaxis],
                use_head=False, 
                line_width=directions_width)
            print(vectors)
            plot += vectors

#             directions_to_plot = np.hstack((samples, samples + directions))
            
#             for i, dir_to_plot in enumerate(directions_to_plot):
#                 dir_to_plot = dir_to_plot.reshape((2, 3))
#                 if np.all(dir_to_plot[0] == dir_to_plot[1]):
#                     continue
#                 color = int(colors[i]) if None is not colors else samples_color
#                 plt_line = k3d.line(dir_to_plot, 
#                                     shader='mesh', width=directions_width, color=color)
#                 plot += plt_line

    if None is not sharp_vert:
        k3d_points = k3d.points(sharp_vert,
                                point_size=sharpvert_psize, color=sharpvert_color)
        plot += k3d_points
        k3d_points.shader='flat'
        
        if None is not sharp_curves:            
            if None is not sharpcurve_color:
                color = sharpcurve_color
            else:
                import randomcolor
                rand_color = randomcolor.RandomColor()
            for i, vert_ind in enumerate(sharp_curves):
                sharp_points_curve = mesh.vertices[vert_ind]
                
                if None is sharpcurve_color:
                    color = rand_color.generate(hue='red')[0]
                    color = int('0x' + color[1:], 16)
                plt_line = k3d.line(sharp_points_curve, 
                                    shader='mesh', width=sharpcurve_width, color=color)
                plot += plt_line
        
    plot.grid_visible = False
    plot.display()
    
    if as_image:
        plot.fetch_screenshot()
        return Image(data=b64decode(plot.screenshot))

In [5]:
# mm/pixel
HIGH_RES = 0.02
MED_RES = 0.05
LOW_RES = 0.125
XLOW_RES = 0.25

DISPLAY_RES = 0.03

In [6]:
base_groundtruth_dir = '/logs/'

base_predictions_dir = '/logs/whole_models_inference/pc-high-baseline/'

obj_name = 'abc_0022_00221435_3b6fccab77e297bb4a84ae22_000'

## Display whole model without predictions 

In [7]:
ground_truth_filename = os.path.join(base_groundtruth_dir, obj_name + '.hdf5')
ground_truth_dataset = Hdf5File(
    ground_truth_filename,
    io=io.WholePointCloudIO,
    preload=PreloadTypes.LAZY,
    labels='*')

ground_truth = [patch for patch in ground_truth_dataset]

In [8]:
n_points = np.concatenate([
    patch['indexes_in_whole']
    for patch in ground_truth]).max() + 1

In [9]:
whole_model_points_gt = np.zeros((n_points, 3))
whole_model_distances_gt = np.ones(n_points) * np.inf
whole_model_directions_gt = np.ones( (n_points, 3) ) * np.inf

In [10]:
for patch in tqdm(ground_truth):
    distances = patch['distances']
    directions = patch['directions'].reshape((-1, 3))
    indexes = patch['indexes_in_whole']
    whole_model_points_gt[indexes] = patch['points'].reshape((-1, 3))

    assign_mask = whole_model_distances_gt[indexes] > distances
    whole_model_distances_gt[indexes[assign_mask]] = np.minimum(distances[assign_mask], 1.0)
    whole_model_directions_gt[indexes[assign_mask]] = directions[assign_mask]

In [11]:
display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
                  samples=whole_model_points_gt, samples_distances=whole_model_distances_gt,
                  samples_color=0x0000ff, samples_psize=DISPLAY_RES,
                  directions=None)

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

## Display model fragment 

In [12]:
# idx = np.random.randint(len(ground_truth))

# patch = ground_truth[idx]

# display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
#                   sharp_vert=None, sharpvert_psize=HIGH_RES,
#                   samples=patch['points'].reshape((-1, 3)), samples_distances=patch['distances'],
#                   samples_color=0x0000ff, samples_psize=0.02,
#                   directions=None)
# # patch['directions'].reshape((-1, 3))

## Load and combine predictions

In [13]:

predictions_filename = os.path.join(base_predictions_dir, obj_name, 'predictions/test_predictions_0.hdf5')

predictions_dataset = Hdf5File(
    predictions_filename,
    io=io.WholePointCloudIO,
    preload=PreloadTypes.LAZY,
    labels='*')

predictions = [patch for patch in predictions_dataset]

File /logs/whole_models_inference/pc-high-baseline/abc_0022_00221435_3b6fccab77e297bb4a84ae22_000/predictions/test_predictions_0.hdf5 is not compatible with Hdf5File I/O interface <class 'sharpf.utils.abc_utils.hdf5.io_struct.HDF5IO'>
File /logs/whole_models_inference/pc-high-baseline/abc_0022_00221435_3b6fccab77e297bb4a84ae22_000/predictions/test_predictions_0.hdf5 is not compatible with Hdf5File I/O interface <class 'sharpf.utils.abc_utils.hdf5.io_struct.HDF5IO'>


In [14]:
from scipy.stats.mstats import mquantiles

class adv(object):
    def __init__(self, probability, func=np.mean):
        self._probability = probability
        self._func = func

    def __call__(self, values):
        quantiles = mquantiles(values,
                               [(1.0 - self._probability) / 2.0, (1.0 + self._probability) / 2.0],
                               alphap=0.5, betap=0.5)
        values = list(filter(lambda value: quantiles[0] <= value <= quantiles[1], values))
        if not values:
            raise AveragingNoDataException('No values after trimming')
        result = self._func(values)
        return result


In [15]:
def combine_predictions_pointwise(predictions, patches, func=np.median):
    n_points = np.concatenate([
        patch['indexes_in_whole']
        for patch in patches]).max() + 1
    
    whole_model_points_pred = np.zeros((n_points, 3))
    whole_model_distances_pred = np.ones(n_points) * np.inf

    d = defaultdict(list)
    for prediction, patch in tqdm(zip(predictions, patches)):
        distances = prediction['distances']
        indexes_gt = patch['indexes_in_whole']
        points_gt = patch['points']
        whole_model_points_pred[indexes_gt] = points_gt.reshape((-1, 3))

        for i, idx in enumerate(indexes_gt):
            d[idx].append(distances[i])
    
    for idx, values in d.items():
        whole_model_distances_pred[idx] = func(values)
        
    return whole_model_points_pred, whole_model_distances_pred, d


def combine_predictions_median(predictions, patches):
    return combine_predictions_pointwise(
        predictions,
        patches, 
        func=np.median)


def combine_predictions_mean(predictions, patches):
    return combine_predictions_pointwise(
        predictions,
        patches, 
        func=np.mean)


def combine_predictions_min(predictions, patches):
    return combine_predictions_pointwise(
        predictions,
        patches, 
        func=np.min)


def combine_predictions_adv(predictions, patches):
    return combine_predictions_pointwise(
        predictions,
        patches, 
        func=adv(0.6))

In [16]:
whole_model_points_pred, whole_model_distances_pred_median, d = \
    combine_predictions_median(predictions, ground_truth)

In [17]:
whole_model_points_pred, whole_model_distances_pred_mean, d = \
    combine_predictions_mean(predictions, ground_truth)

In [18]:
whole_model_points_pred, whole_model_distances_pred_min, d = \
    combine_predictions_min(predictions, ground_truth)

In [19]:
whole_model_points_pred, whole_model_distances_pred_adv60, d = \
    combine_predictions_adv(predictions, ground_truth)

In [20]:
def combine_predictions_pointwise_mean_of_mins(predictions, patches):
    n_points = np.concatenate([
        patch['indexes_in_whole']
        for patch in patches]).max() + 1
    
    whole_model_points_pred = np.zeros((n_points, 3))
    whole_model_distances_pred = np.ones(n_points) * np.inf

    d = defaultdict(list)
    for prediction, patch in tqdm(zip(predictions, patches)):
        distances = prediction['distances']
        indexes_gt = patch['indexes_in_whole']
        points_gt = patch['points']
        whole_model_points_pred[indexes_gt] = points_gt.reshape((-1, 3))

        for i, idx in enumerate(indexes_gt):
            d[idx].append(distances[i])

    d = {idx: np.array(values) for idx, values in d.items()}
    
    for idx, values in d.items():
        whole_model_distances_pred[idx] = np.mean(values[values < 0.9])
        
    return whole_model_points_pred, whole_model_distances_pred, d

In [21]:
whole_model_points_pred, whole_model_distances_pred_mom, d = \
    combine_predictions_adv(predictions, ground_truth)

In [22]:
def combine_predictions_pointwise_noborder(predictions, patches, brd_thr=90, func=np.median):
    n_points = np.concatenate([
        patch['indexes_in_whole']
        for patch in patches]).max() + 1
    
    whole_model_points_pred = np.zeros((n_points, 3))
    whole_model_distances_pred = np.ones(n_points) * np.inf

    d = defaultdict(list)
    for prediction, patch in tqdm(zip(predictions, patches)):
        distances = prediction['distances']
        indexes_gt = patch['indexes_in_whole']
        points_gt = patch['points'].reshape((-1, 3))
        whole_model_points_pred[indexes_gt] = points_gt
        
        points_radii = np.linalg.norm(
                points_gt - points_gt.mean(axis=0),
                axis=1)
        center_indexes = np.where(points_radii < np.percentile(points_radii, brd_thr))[0]

        for i, idx in enumerate(center_indexes):
            d[indexes_gt[idx]].append(distances[center_indexes[i]])

    d = {idx: np.array(values) for idx, values in d.items()}
    
    for idx, values in d.items():
        whole_model_distances_pred[idx] = func(values)
        
    return whole_model_points_pred, whole_model_distances_pred, d

In [23]:
whole_model_points_pred, whole_model_distances_pred_noborder, d = \
    combine_predictions_pointwise_noborder(predictions, ground_truth, brd_thr=80, func=np.min)

In [24]:
def combine_predictions_pointwise_snap(
    predictions,
    patches, 
    brd_thr=90,
    snap_thr=.99,
    func=np.median):
    
    n_points = np.concatenate([
        patch['indexes_in_whole']
        for patch in patches]).max() + 1
    
    whole_model_points_pred = np.zeros((n_points, 3))
    whole_model_distances_pred = np.ones(n_points) * np.inf

    d = defaultdict(list)
    for prediction, patch in tqdm(zip(predictions, patches)):
        distances = prediction['distances']
        indexes_gt = patch['indexes_in_whole']
        points_gt = patch['points'].reshape((-1, 3))
        whole_model_points_pred[indexes_gt] = points_gt

        points_radii = np.linalg.norm(
                points_gt - points_gt.mean(axis=0),
                axis=1)
        center_indexes = np.where(points_radii < np.percentile(points_radii, brd_thr))[0]

        for i, idx in enumerate(center_indexes):
            d[indexes_gt[idx]].append(distances[center_indexes[i]])

    d = {idx: np.array(values) for idx, values in d.items()}
    
    for idx, values in d.items():
        if np.sum(values > snap_thr) * 2 > len(values):
            whole_model_distances_pred[idx]
        else:
            whole_model_distances_pred[idx] = func(values)
        
    return whole_model_points_pred, whole_model_distances_pred, d

In [25]:
whole_model_points_pred, whole_model_distances_pred_snap, d = \
    combine_predictions_pointwise_snap(predictions, ground_truth, brd_thr=80, snap_thr=0.9, func=np.min)

In [26]:
whole_model_distances_pred = whole_model_distances_pred_noborder


display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
                  samples=whole_model_points_pred,
                  samples_distances=whole_model_distances_pred,
                  samples_color=0x0000ff, samples_psize=DISPLAY_RES,
                  directions=None)

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [27]:
# predictions_abs_diff = np.abs(whole_model_distances_gt - whole_model_distances_pred)


# display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
#                   samples=whole_model_points_pred, samples_distances=predictions_abs_diff,
#                   samples_color=0x0000ff, samples_psize=DISPLAY_RESHIGH_RES,
#                   directions=None,
#                   cmap=k3d.colormaps.matplotlib_color_maps.coolwarm)

In [28]:
from scipy.spatial import cKDTree

In [29]:
n_omp_threads = int(os.environ.get('OMP_NUM_THREADS', 1))
nn_distances, nn_indexes = cKDTree(whole_model_points_pred, leafsize=100)\
    .query(whole_model_points_pred, k=51, n_jobs=36)

In [30]:
import torch
from torch import optim

In [31]:
def l2_knn_smoothing_loss(predictions, init_predictions, nn_indexes, alpha):
    data_fidelity_term = (predictions - init_predictions) ** 2
    regularization_term = torch.sum(
        (predictions[nn_indexes[:, 1:]] - 
         predictions.reshape((len(predictions), 1))) ** 2,
        dim=1)
    return torch.sum(data_fidelity_term) + alpha * torch.sum(regularization_term)

In [32]:
init_predictions_th = torch.Tensor(whole_model_distances_pred_min)

predictions_th = torch.ones(whole_model_distances_pred_min.shape)
predictions_th.requires_grad_()

optimizer = optim.SGD([predictions_th], lr=0.001, momentum=0.9)

In [33]:
t = trange(300, desc='Optimization', leave=True)
for i in t:
    optimizer.zero_grad()
    loss = l2_knn_smoothing_loss(predictions_th, init_predictions_th, nn_indexes, 0.01)
    loss.backward()
    optimizer.step()
    s = 'Optimization: step #{0:}, loss: {1:3.1f}'.format(i, loss.item())
    t.set_description(s)
    t.refresh() 

In [34]:
whole_model_distances_pred_knn5_l2 = predictions_th.detach().numpy()

In [35]:
display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
                  samples=whole_model_points_pred, 
                  samples_distances=whole_model_distances_pred_knn5_l2,
                  samples_color=0x0000ff, samples_psize=DISPLAY_RES,
                  directions=None)

Output()

In [36]:
def tv_knn_smoothing_loss(predictions, init_predictions, nn_indexes, alpha):
    data_fidelity_term = (predictions - init_predictions) ** 2
    regularization_term = torch.sum(
        torch.abs(
            predictions[nn_indexes[:, 1:]] - 
            predictions.reshape((len(predictions), 1))),
        dim=1)
    return torch.sum(data_fidelity_term) + alpha * torch.sum(regularization_term)

In [37]:
init_predictions_th = torch.Tensor(whole_model_distances_pred_min)

predictions_th = torch.ones(whole_model_distances_pred_min.shape)
predictions_th.requires_grad_()

optimizer = optim.SGD([predictions_th], lr=0.001, momentum=0.9)

In [38]:
t = trange(300, desc='Optimization', leave=True)
for i in t:
    optimizer.zero_grad()
    loss = tv_knn_smoothing_loss(predictions_th, init_predictions_th, nn_indexes, 0.001)
    loss.backward()
    optimizer.step()
    s = 'Optimization: step #{0:}, loss: {1:3.1f}'.format(i, loss.item())
    t.set_description(s)
    t.refresh() 

In [39]:
whole_model_distances_pred_knn5_tv = predictions_th.detach().numpy()

In [40]:
display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
                  samples=whole_model_points_pred, 
                  samples_distances=whole_model_distances_pred_knn5_tv,
                  samples_color=0x0000ff, samples_psize=DISPLAY_RES,
                  directions=None)

Output()

In [41]:
def make_xy(idx, points, nn_indexes, pred_patches):
    indexes = nn_indexes[idx]
    
    X, y = [], []
    for i in indexes:
        for y_value in pred_patches[i]:
            X.append(points[i])
            y.append(y_value)
    
    return np.array(X), np.array(y), np.unique(X, axis=0, return_index=True)[1]


def data_maker(points, nn_indexes, pred_patches):
    for idx in range(len(points)):
        X, y, uniq_indexes = make_xy(idx, points, nn_indexes, pred_patches)
        yield idx, X, y, uniq_indexes

In [42]:
from sklearn.decomposition import PCA
from sklearn.linear_model import (
    LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor)
from sklearn.preprocessing import PolynomialFeatures


def local_linear_fit_smoothing(idx, X, y, uniq_indexes):
    X_trans = PCA(n_components=2).fit_transform(X)
    X_trans = PolynomialFeatures(2).fit_transform(X_trans)

    huber_alpha_opt = 4.
    huber_epsilon_opt = 1.
    estimator = HuberRegressor(epsilon=huber_epsilon_opt, alpha=huber_alpha_opt)
#     estimator = RANSACRegressor(
#         HuberRegressor(epsilon=huber_epsilon_opt, alpha=huber_alpha_opt),
#         random_state=0)

    try:    
        y_pred = estimator.fit(X_trans, y).predict(X_trans)
    except ValueError:
        y_pred = None

    return y_pred

In [43]:
n_omp_threads = int(os.environ.get('OMP_NUM_THREADS', 1))
nn_distances, nn_indexes = cKDTree(whole_model_points_pred, leafsize=100)\
    .query(whole_model_points_pred, k=51, n_jobs=36)

In [44]:
parallel = Parallel(n_jobs=32, backend='loky', verbose=100)
delayed_iterable = (delayed(local_linear_fit_smoothing)(idx, X, y, uniq_indexes)
    for idx, X, y, uniq_indexes in data_maker(whole_model_points_gt, nn_indexes, d))
result = parallel(delayed_iterable)

[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   1 tasks      | elapsed:    1.0s
[Parallel(n_jobs=32)]: Done   2 tasks      | elapsed:    1.1s
[Parallel(n_jobs=32)]: Done   3 tasks      | elapsed:    1.1s
[Parallel(n_jobs=32)]: Done   4 tasks      | elapsed:    1.1s
[Parallel(n_jobs=32)]: Done   5 tasks      | elapsed:    1.1s
[Parallel(n_jobs=32)]: Done   6 tasks      | elapsed:    1.1s
[Parallel(n_jobs=32)]: Done   7 tasks      | elapsed:    1.1s
[Parallel(n_jobs=32)]: Done   8 tasks      | elapsed:    1.1s
[Parallel(n_jobs=32)]: Done   9 tasks      | elapsed:    1.1s
[Parallel(n_jobs=32)]: Done  10 tasks      | elapsed:    1.1s
[Parallel(n_jobs=32)]: Done  11 tasks      | elapsed:    1.1s
[Parallel(n_jobs=32)]: Done  12 tasks      | elapsed:    1.1s
[Parallel(n_jobs=32)]: Done  13 tasks      | elapsed:    1.1s
[Parallel(n_jobs=32)]: Done  14 tasks      | elapsed:    1.1s
[Parallel(n_jobs=32)]: Done  15 tasks      | elapsed:  

[Parallel(n_jobs=32)]: Done 162 tasks      | elapsed:    1.7s
[Parallel(n_jobs=32)]: Done 163 tasks      | elapsed:    1.7s
[Parallel(n_jobs=32)]: Done 164 tasks      | elapsed:    1.7s
[Parallel(n_jobs=32)]: Done 165 tasks      | elapsed:    1.7s
[Parallel(n_jobs=32)]: Done 166 tasks      | elapsed:    1.7s
[Parallel(n_jobs=32)]: Done 167 tasks      | elapsed:    1.7s
[Parallel(n_jobs=32)]: Done 168 tasks      | elapsed:    1.7s
[Parallel(n_jobs=32)]: Done 169 tasks      | elapsed:    1.7s
[Parallel(n_jobs=32)]: Done 170 tasks      | elapsed:    1.7s
[Parallel(n_jobs=32)]: Done 171 tasks      | elapsed:    1.7s
[Parallel(n_jobs=32)]: Done 172 tasks      | elapsed:    1.7s
[Parallel(n_jobs=32)]: Done 173 tasks      | elapsed:    1.7s
[Parallel(n_jobs=32)]: Done 174 tasks      | elapsed:    1.7s
[Parallel(n_jobs=32)]: Done 175 tasks      | elapsed:    1.7s
[Parallel(n_jobs=32)]: Done 176 tasks      | elapsed:    1.7s
[Parallel(n_jobs=32)]: Done 177 tasks      | elapsed:    1.7s
[Paralle

[Parallel(n_jobs=32)]: Done 584 tasks      | elapsed:    2.8s
[Parallel(n_jobs=32)]: Done 588 tasks      | elapsed:    2.8s
[Parallel(n_jobs=32)]: Done 592 tasks      | elapsed:    2.8s
[Parallel(n_jobs=32)]: Done 596 tasks      | elapsed:    2.8s
[Parallel(n_jobs=32)]: Done 600 tasks      | elapsed:    2.8s
[Parallel(n_jobs=32)]: Done 604 tasks      | elapsed:    2.8s
[Parallel(n_jobs=32)]: Done 608 tasks      | elapsed:    2.8s
[Parallel(n_jobs=32)]: Done 612 tasks      | elapsed:    2.8s
[Parallel(n_jobs=32)]: Done 616 tasks      | elapsed:    2.8s
[Parallel(n_jobs=32)]: Done 620 tasks      | elapsed:    2.8s
[Parallel(n_jobs=32)]: Done 624 tasks      | elapsed:    2.8s
[Parallel(n_jobs=32)]: Done 628 tasks      | elapsed:    2.8s
[Parallel(n_jobs=32)]: Done 632 tasks      | elapsed:    2.8s
[Parallel(n_jobs=32)]: Done 636 tasks      | elapsed:    2.8s
[Parallel(n_jobs=32)]: Done 640 tasks      | elapsed:    2.8s
[Parallel(n_jobs=32)]: Done 644 tasks      | elapsed:    2.8s
[Paralle

[Parallel(n_jobs=32)]: Done 1224 tasks      | elapsed:    4.0s
[Parallel(n_jobs=32)]: Done 1228 tasks      | elapsed:    4.0s
[Parallel(n_jobs=32)]: Done 1232 tasks      | elapsed:    4.0s
[Parallel(n_jobs=32)]: Done 1236 tasks      | elapsed:    4.0s
[Parallel(n_jobs=32)]: Done 1240 tasks      | elapsed:    4.0s
[Parallel(n_jobs=32)]: Done 1244 tasks      | elapsed:    4.0s
[Parallel(n_jobs=32)]: Done 1248 tasks      | elapsed:    4.0s
[Parallel(n_jobs=32)]: Done 1252 tasks      | elapsed:    4.0s
[Parallel(n_jobs=32)]: Done 1256 tasks      | elapsed:    4.0s
[Parallel(n_jobs=32)]: Done 1260 tasks      | elapsed:    4.0s
[Parallel(n_jobs=32)]: Done 1264 tasks      | elapsed:    4.0s
[Parallel(n_jobs=32)]: Done 1268 tasks      | elapsed:    4.0s
[Parallel(n_jobs=32)]: Done 1272 tasks      | elapsed:    4.0s
[Parallel(n_jobs=32)]: Done 1276 tasks      | elapsed:    4.0s
[Parallel(n_jobs=32)]: Done 1280 tasks      | elapsed:    4.0s
[Parallel(n_jobs=32)]: Done 1284 tasks      | elapsed: 

[Parallel(n_jobs=32)]: Done 1864 tasks      | elapsed:    5.1s
[Parallel(n_jobs=32)]: Done 1868 tasks      | elapsed:    5.1s
[Parallel(n_jobs=32)]: Done 1872 tasks      | elapsed:    5.1s
[Parallel(n_jobs=32)]: Done 1876 tasks      | elapsed:    5.1s
[Parallel(n_jobs=32)]: Done 1880 tasks      | elapsed:    5.1s
[Parallel(n_jobs=32)]: Done 1884 tasks      | elapsed:    5.2s
[Parallel(n_jobs=32)]: Done 1888 tasks      | elapsed:    5.2s
[Parallel(n_jobs=32)]: Done 1892 tasks      | elapsed:    5.2s
[Parallel(n_jobs=32)]: Done 1896 tasks      | elapsed:    5.2s
[Parallel(n_jobs=32)]: Done 1900 tasks      | elapsed:    5.2s
[Parallel(n_jobs=32)]: Done 1904 tasks      | elapsed:    5.2s
[Parallel(n_jobs=32)]: Done 1908 tasks      | elapsed:    5.2s
[Parallel(n_jobs=32)]: Done 1912 tasks      | elapsed:    5.2s
[Parallel(n_jobs=32)]: Done 1916 tasks      | elapsed:    5.2s
[Parallel(n_jobs=32)]: Done 1920 tasks      | elapsed:    5.2s
[Parallel(n_jobs=32)]: Done 1924 tasks      | elapsed: 

...

[Parallel(n_jobs=32)]: Done 84936 tasks      | elapsed:  2.9min
[Parallel(n_jobs=32)]: Done 84940 tasks      | elapsed:  2.9min
[Parallel(n_jobs=32)]: Done 84944 tasks      | elapsed:  2.9min
[Parallel(n_jobs=32)]: Done 84948 tasks      | elapsed:  2.9min
[Parallel(n_jobs=32)]: Done 84952 tasks      | elapsed:  2.9min
[Parallel(n_jobs=32)]: Done 84956 tasks      | elapsed:  2.9min
[Parallel(n_jobs=32)]: Done 84960 tasks      | elapsed:  2.9min
[Parallel(n_jobs=32)]: Done 84964 tasks      | elapsed:  2.9min
[Parallel(n_jobs=32)]: Done 84968 tasks      | elapsed:  2.9min
[Parallel(n_jobs=32)]: Done 84972 tasks      | elapsed:  2.9min
[Parallel(n_jobs=32)]: Done 84976 tasks      | elapsed:  2.9min
[Parallel(n_jobs=32)]: Done 84980 tasks      | elapsed:  2.9min
[Parallel(n_jobs=32)]: Done 84984 tasks      | elapsed:  2.9min
[Parallel(n_jobs=32)]: Done 84988 tasks      | elapsed:  2.9min
[Parallel(n_jobs=32)]: Done 84992 tasks      | elapsed:  2.9min
[Parallel(n_jobs=32)]: Done 84996 tasks 

[Parallel(n_jobs=32)]: Done 85576 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 85580 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 85584 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 85588 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 85592 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 85596 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 85600 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 85604 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 85608 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 85612 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 85616 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 85620 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 85624 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 85628 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 85632 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 85636 tasks 

[Parallel(n_jobs=32)]: Done 86216 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86220 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86224 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86228 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86232 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86236 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86240 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86244 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86248 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86252 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86256 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86260 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86264 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86268 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86272 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86276 tasks 

[Parallel(n_jobs=32)]: Done 86856 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86860 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86864 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86868 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86872 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86876 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86880 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86884 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86888 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86892 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86896 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86900 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86904 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86908 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86912 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 86916 tasks 

[Parallel(n_jobs=32)]: Done 87496 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 87500 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 87504 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 87508 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 87512 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 87516 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 87520 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 87524 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 87528 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 87532 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 87536 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 87540 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 87544 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 87548 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 87552 tasks      | elapsed:  3.0min
[Parallel(n_jobs=32)]: Done 87556 tasks 

In [45]:
d_out = defaultdict(list)

for y_pred, (idx, X, y, uniq_indexes) in tqdm(
        zip(result, data_maker(whole_model_points_gt, nn_indexes, d))):
    if None is y_pred:
        continue
    for i, nn_index in enumerate(nn_indexes[idx]):
        d_out[nn_index].append(y_pred[uniq_indexes[i]])


In [46]:
whole_model_distances_pred_lin_fit = np.zeros(len(d_out))

for idx, values in d_out.items():
    whole_model_distances_pred_lin_fit[idx] = np.mean(values)


In [47]:
display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
                  samples=whole_model_points_gt, 
                  samples_distances=whole_model_distances_pred_lin_fit,
                  samples_color=0x0000ff, samples_psize=DISPLAY_RES,
                  directions=None)

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [48]:
from collections import Callable, Mapping
from copy import deepcopy
from typing import Tuple, MutableMapping

import numpy as np


class Metric(Callable):
    def __call__(self, true_instance, pred_label):
        pass


class PointwiseSquaredError(Metric):
    """Points with error above threshold are considered bad."""
    def __call__(self, true_instance, pred_label):
        true_distances = true_instance['distances']
        pred_distances = pred_label['distances']
        diff_sq = np.square(true_distances - pred_distances)
        return diff_sq


class RMSE(Metric):
    def __init__(self, normalize=False):
        self._normalize = normalize

    def __call__(self, true_instance, pred_label):
        diff_sq = PointwiseSquaredError()(true_instance, pred_label)
        rmse = np.sqrt(
            np.mean(diff_sq))
        
        if self._normalize:
            rmse /= len(diff_sq)

        return rmse

    def __str__(self): 
        return 'RMSE'


class BadPoints(Metric):
    """Points with error above threshold are considered bad."""
    def __init__(self, threshold, normalize=False):
        self._threshold_sq = threshold ** 2
        self._normalize = normalize
        
    def __str__(self): 
        return 'Bad {0:3.3g}'.format(np.sqrt(self._threshold_sq))

    def __call__(self, true_instance, pred_label):
        diff_sq = PointwiseSquaredError()(true_instance, pred_label)
        bad_points = np.sum(diff_sq > self._threshold_sq)
        
        if self._normalize:
            bad_points /= len(diff_sq)
        
        return bad_points


class RMSEQuantile(Metric):
    """Determine what is error level for most points."""
    def __init__(self, proba, normalize=False):
        self._proba = proba
        self._normalize = normalize
        
    def __str__(self): 
        return 'Q {0:3.3g}'.format(self._proba)

    def __call__(self, true_instance, pred_label):
        diff_sq = PointwiseSquaredError()(true_instance, pred_label)
        q = np.quantile(diff_sq, self._proba)
        return q


class PointwiseMaskSelector(Callable):
    def __call__(self, true_instance: MutableMapping, pred_label: MutableMapping) \
            -> Tuple[MutableMapping, MutableMapping]:
        pass


class DistanceLessThan(PointwiseMaskSelector):
    def __init__(self, threshold):
        self._threshold = threshold

    def __call__(self, true_instance, pred_label):
        masked_true_instance = deepcopy(true_instance)
        true_distances = masked_true_instance['distances']
        selection_mask = true_distances < self._threshold
        masked_true_instance['distances'] = true_distances[selection_mask]

        masked_pred_label = deepcopy(pred_label)
        pred_distances = masked_pred_label['distances']
        masked_pred_label['distances'] = pred_distances[selection_mask]
        return masked_true_instance, masked_pred_label

    
class MaskedMetric(Metric):
    def __init__(self, mask_threshold, metric):
        self._metric = metric
        self._masking = DistanceLessThan(mask_threshold)
        
    def __str__(self): 
        return 'M{}'.format(str(self._metric))

    def __call__(self, true_instance, pred_label):
        true_instance, pred_label = self._masking(true_instance, pred_label)
        return self._metric(true_instance, pred_label)


class RescaledMetric(Metric):
    def __init__(self, scale_factor, metric):
        self._metric = metric
        self._scale_factor = scale_factor
        
    def __str__(self): 
        return '{}'.format(str(self._metric))

    def __call__(self, true_instance, pred_label):
        value = self._metric(true_instance, pred_label)
        return value * self._scale_factor


In [49]:
resolution_3d = HIGH_RES

rmse_scale = 1e6
rmseq_scale = 1e3
mask_thr = 1.0

rmse = RescaledMetric(rmse_scale, RMSE(normalize=True))
masked_rmse = MaskedMetric(mask_thr, rmse)

bad_points_1r = BadPoints(1. * resolution_3d, normalize=True)
bad_points_2r = BadPoints(2. * resolution_3d, normalize=True)
bad_points_4r = BadPoints(4. * resolution_3d, normalize=True)
bad_points_8r = BadPoints(8. * resolution_3d, normalize=True)
bad_points_16r = BadPoints(16. * resolution_3d, normalize=True)

masked_bad_points_1r = MaskedMetric(mask_thr, bad_points_1r)
masked_bad_points_2r = MaskedMetric(mask_thr, bad_points_2r)
masked_bad_points_4r = MaskedMetric(mask_thr, bad_points_4r)
masked_bad_points_8r = MaskedMetric(mask_thr, bad_points_8r)
masked_bad_points_16r = MaskedMetric(mask_thr, bad_points_16r)

rmse_q90 = RescaledMetric(rmseq_scale, RMSEQuantile(0.9))
rmse_q95 = RescaledMetric(rmseq_scale, RMSEQuantile(0.95))
rmse_q99 = RescaledMetric(rmseq_scale, RMSEQuantile(0.99))

masked_rmse_q90 = MaskedMetric(mask_thr, rmse_q90)
masked_rmse_q95 = MaskedMetric(mask_thr, rmse_q95)
masked_rmse_q99 = MaskedMetric(mask_thr, rmse_q99)

metrics = [
    rmse,
    masked_rmse,
    bad_points_1r,
#     bad_points_2r,
    bad_points_4r,
#     bad_points_8r,
#     bad_points_16r,
    masked_bad_points_1r,
#     masked_bad_points_2r,
    masked_bad_points_4r,
#     masked_bad_points_8r,
#     masked_bad_points_16r,
#     rmse_q90,
    rmse_q95,
#     rmse_q99,
#     masked_rmse_q90,
    masked_rmse_q95,
#     masked_rmse_q99,
]

In [50]:
predictions_by_name = {
    'adv60': whole_model_distances_pred_adv60,
    'median': whole_model_distances_pred_median,
    'mean': whole_model_distances_pred_mean,
    'mom': whole_model_distances_pred_mom,
    'min': whole_model_distances_pred_min,
    'nobrd': whole_model_distances_pred_noborder,
    'snap': whole_model_distances_pred_snap,
    'knn-l2': whole_model_distances_pred_knn5_l2,
    'knn-tv': whole_model_distances_pred_knn5_tv,
    'rlrfit': whole_model_distances_pred_lin_fit
}

metric_values_by_name = defaultdict(list)
for name, values in predictions_by_name.items():
    for m in metrics:
        value = m({'distances': whole_model_distances_gt}, {'distances': values})
        metric_values_by_name[name].append(value)

In [51]:
header = 'Method'.rjust(6) + ' '.join([str(m).rjust(9) for m in metrics])
horizontal_line = '-' * len(header)
data_str = [
    name.rjust(6) + ' '.join(['{0:3.3g}'.format(value).rjust(9) for value in metric_values])
    for name, metric_values in metric_values_by_name.items()
]

print('\n'.join([header, horizontal_line] + data_str))

Method     RMSE     MRMSE  Bad 0.02  Bad 0.08 MBad 0.02 MBad 0.08    Q 0.95   MQ 0.95
-------------------------------------------------------------------------------------
 adv60    0.932     0.942     0.581      0.26     0.585     0.262      33.4      33.5
median    0.841     0.851      0.51     0.162     0.514     0.164      30.6      30.8
  mean     1.17      1.18     0.718     0.433     0.723     0.437        42      42.2
   mom    0.932     0.942     0.581      0.26     0.585     0.262      33.4      33.5
   min    0.903     0.913     0.543     0.171     0.547     0.172      20.7      20.8
 nobrd    0.824     0.834     0.517     0.146     0.521     0.147      18.5      18.6
  snap      inf       inf     0.531     0.178     0.527     0.172      63.4      44.1
knn-l2    0.876     0.886     0.573     0.164     0.576     0.166      18.6      18.7
knn-tv    0.898     0.908     0.602     0.174     0.606     0.175      19.7      19.8
rlrfit    0.766     0.775     0.522     0.148     0.52

#### garbage

In [52]:
# std_values = np.zeros(len(whole_model_distances_gt))

In [53]:
# for idx, values in d.items():
#     std_values[idx] = np.std(values) 

In [54]:
# alpha = 1e0

# w_sum = np.sum(
#     1. / std_values[nn_indexes[:, 1:]] * whole_model_distances_pred_min[nn_indexes[:, 1:]],
#     axis=1)

# coef_sum = np.sum(
#     1. / std_values[nn_indexes[:, 1:]],
#     axis=1)

# whole_model_distances_pred_knn5 = (whole_model_distances_pred_min + alpha * w_sum) / (1 + alpha * coef_sum)

In [55]:
# whole_model_distances_pred_knn5 - whole_model_distances_pred_min

In [56]:
# len(ground_truth) * 4096

In [57]:
# idx = np.random.randint(len(predictions))
# patch = ground_truth[idx]
# patch_pred = predictions[idx]

# points = patch['points'].reshape((-1, 3))
# distances = patch_pred['distances']
# # center_indexes = np.argsort(
# #     np.linalg.norm(
# #         points - points.mean(axis=0),
# #         axis=1))
# # center_indexes = center_indexes[:-1000]

# points_radii = np.linalg.norm(
#         points - points.mean(axis=0),
#         axis=1)
# center_indexes = np.where(points_radii < np.percentile(points_radii, 70))

# display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
#                   sharp_vert=points[center_indexes],
#                   sharpvert_psize=MED_RES,
#                   samples=points, 
#                   samples_distances=distances,
#                   samples_color=0x0000ff, samples_psize=DISPLAY_RES,
#                   directions=None)

#### patch inspection

In [58]:
# idx = np.random.choice(len(whole_model_distances_gt))
# sorted(d[idx]), whole_model_distances_gt[idx], idx

In [59]:
# def get_patches_with_index(patches, index):
#     patches_with_index = []
#     indexes = []
#     for i, patch in enumerate(patches):
#         if index in patch['indexes_in_whole']:
#             patches_with_index.append(patch)
#             indexes.append(i)
#     return patches_with_index, indexes


In [60]:
# patches_with_index, indexes = get_patches_with_index(ground_truth, 86585)

In [61]:
# indexes

In [62]:
# idx = 5
# patch = patches_with_index[idx]
# patch_predictions = predictions[indexes[idx]]


# print(patch_predictions['distances'][patch['indexes_in_whole'] == 86585])

# display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
#                   sharp_vert=whole_model_points_gt[86585:86585+1], sharpvert_psize=MED_RES,
#                   samples=patch['points'].reshape((-1, 3)), samples_distances=patch_predictions['distances'],
#                   samples_color=0x0000ff, samples_psize=DISPLAY_RES,
#                   directions=None)

In [63]:
# idx = 1
# X = whole_model_points_gt[nn_indexes[idx]]

In [64]:
# display_sharpness(None, plot_meshvert=False, meshvert_psize=0.01,
#                   samples=X, 
#                   samples_distances=whole_model_distances_pred_min[nn_indexes[idx]],
#                   samples_color=0x0000ff, samples_psize=DISPLAY_RES,
#                   directions=None)